In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot as plt
from pandas import Series
from sklearn.metrics import mean_squared_error
from datetime import datetime
from ipywidgets import IntProgress

/home/glissa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [2]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [3]:
import pickle

In [4]:
from tqdm import tqdm_notebook

In [5]:
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [6]:
INPUT_PATH = '../../../data/train_test'
OUTPUT_PATH = '../../../data/train_test'
DAYS_PRED = 28

### FUNCTIONS

In [7]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [8]:
# score a model, return None on failure
def score_model(train, cfg, debug=False):
    result = None
    

    m = Prophet(**cfg)


    # show all warnings and fail on exception if debugging
    if debug:
        m.fit(train)

        future = m.make_future_dataframe(periods=DAYS_PRED)

        forecast = m.predict(future)

        y_hat = forecast.iloc[:,-1:]

        y_hat = [i[0] for i in y_hat.values]


    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")

                m.fit(train)
                future = m.make_future_dataframe(periods=DAYS_PRED)

                forecast = m.predict(future)

                y_hat = forecast.iloc[:,-1:]

                y_hat = [i[0] for i in y_hat.values]
        except:
            y_hat = None
    
        
        
    return y_hat

In [9]:
# score a model, return None on failure
def score_model(train, cfg, period=DAYS_PRED):
    result = None
    

    m = Prophet(**cfg)


    # show all warnings and fail on exception if debugging
    with catch_warnings():
        filterwarnings("ignore")

        m.fit(train)
        future = m.make_future_dataframe(periods=period)

        forecast = m.predict(future)


    return forecast

### LOAD DATASET

In [10]:
X_train = pd.read_pickle(f'{INPUT_PATH}/X_train_agg.pkl')
X_val = pd.read_pickle(f'{INPUT_PATH}/X_val_agg.pkl')
y_train = pd.read_pickle(f'{INPUT_PATH}/Y_train_agg.pkl')
y_val = pd.read_pickle(f'{INPUT_PATH}/Y_val_agg.pkl')

In [11]:
X_train = X_train.join(y_train)
X_val = X_val.join(y_val)

In [12]:
level = 'state_id'
features = ['date', 'demand']
y_true = 'demand'

## TRAIN MODEL

In [13]:
cfg_prophet = dict()

In [14]:
cfg_prophet['yearly_seasonality'] = True
cfg_prophet['daily_seasonality'] = False
cfg_prophet['seasonality_prior_scale'] = 0.1

In [15]:
features_prophet = ['ds', 'trend', 'yhat_lower', 'yhat_upper', 'weekly',  'yearly',  'yhat']

In [17]:
forecast_train = pd.DataFrame()
forecast_val = pd.DataFrame()
errors = []
for idx in tqdm_notebook(X_train[level].unique()):
    train = X_train[X_train[level] == idx][features]
    test = X_val[X_val[level] == idx][y_true].tolist()
    

    train.columns = ['ds', 'y']
    
    forecast = score_model(train, cfg_prophet)
    forecast = forecast[features_prophet]
    forecast[level] = idx 
    forecast_train = forecast_train.append(forecast[:-DAYS_PRED])
    forecast_val = forecast_val.append(forecast[-DAYS_PRED:])
      
   
    
    errors.append(rmse(test,forecast.iloc[-DAYS_PRED:,-1]))


In [18]:
errors

[1.5009797714424193, 0.2662139920592784, 0.6217643243638389]

In [19]:
del X_train, X_val, y_train, y_val

### RE-LOAD DATASET

In [20]:
X_train = pd.read_pickle(f'{INPUT_PATH}/X_train.pkl')
X_val = pd.read_pickle(f'{INPUT_PATH}/X_val.pkl')
y_train = pd.read_pickle(f'{INPUT_PATH}/Y_train.pkl')
y_val = pd.read_pickle(f'{INPUT_PATH}/Y_val.pkl')

### MERGE DATASET

In [22]:
X_train = pd.merge(X_train, forecast_train, how='inner', left_on=['state_id', 'date'], right_on=['state_id', 'ds'])

In [23]:
X_val = pd.merge(X_val, forecast_val, how='inner', left_on=['state_id', 'date'], right_on=['state_id', 'ds'])

### SAVE DATASET

In [29]:
X_train.drop(['date','ds'], axis=1, inplace=True)

In [30]:
X_val.drop(['date','ds'], axis=1, inplace=True)

In [31]:
pickle.dump(X_train,      open(f'{OUTPUT_PATH}/X_train_prophet.pkl', 'wb'))
pickle.dump(X_val,        open(f'{OUTPUT_PATH}/X_val_prophet.pkl',   'wb'))
pickle.dump(y_train,      open(f'{OUTPUT_PATH}/Y_train_prophet.pkl', 'wb'))
pickle.dump(y_val,        open(f'{OUTPUT_PATH}/Y_val_prophet.pkl',   'wb'))
